In [26]:
# VideoFileClip 은 비디오 파일을 읽어올 때 사용되는 클래스 
from moviepy.editor import VideoFileClip
# ipython_display() 는 동영상을 주피터 노트북에 렌더링할 수 있게 도와주는 함수
from moviepy.editor import ipython_display

In [27]:
import cv2

#동영상 파일 열기 
vc = cv2.VideoCapture('love2.mp4')

# vc의 get() 함수로 읽을 수 있습니다. FRAME_COUNT 은 비디오 전체 프레임 개수
vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
print (vlen) # video length

for i in range(vlen):
    # ret 은 read() 함수에서 이미지가 반환되면 True, 반대의 경우 False
    ret, img = vc.read()
    if ret == False:
        break
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('Face Detection', cv2.resize(img, None, fx=1.2, fy=1.2))
    # waitKey() 함수는 키보드가 입력될 때 키보드 값을 반환 27=esc
    if cv2.waitKey(1) == 27:
            break

            
vc.release() 
cv2.destroyAllWindows() #열린 창을 모두 닫음
 

54


In [28]:
# addsticker.py
import dlib
import cv2

def img2sticker_orig(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

    # detector
    dlib_rects = detector_hog(img_rgb, 0)
    if len(dlib_rects) < 1:
        return img_orig

    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)

    # head coord
    for dlib_rect, landmark in zip(dlib_rects, list_landmarks):
        x = landmark[30][0] # nose
        y = landmark[30][1] - dlib_rect.width()//2
        w = dlib_rect.width()
        h = dlib_rect.width()
        break

    # sticker
    img_sticker = cv2.resize(img_sticker, (w,h), interpolation=cv2.INTER_NEAREST)
    #스티커 붙일 위치 
    refined_x = x - w // 2
    refined_y = y - h

    if refined_y < 0:
        img_sticker = img_sticker[-refined_y:]
        refined_y = 0

    img_bgr = img_orig.copy()
    #sticker_area는 번역 이미지에서 스티커를 적용할 수 있는 위치
    sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]

    img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] = \
        cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)

    return img_bgr

In [29]:
detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

vc = cv2.VideoCapture('love2.mp4')
img_sticker = cv2.imread('king.png')

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
print (vlen) # 비디오 프레임의 총 개수

54


In [30]:
for i in range(vlen):
    ret, img = vc.read()
    if ret == False:
        break

    ## 추가된 부분
    start = cv2.getTickCount()
    img_result = img2sticker_orig(img, img_sticker.copy(), detector_hog, landmark_predictor)
    time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
    print ('[INFO] time: %.2fms'%time)

    cv2.imshow('show', img_result)
    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.destroyAllWindows()

[INFO] time: 94.11ms
[INFO] time: 125.45ms
[INFO] time: 116.32ms
[INFO] time: 66.63ms
[INFO] time: 69.92ms
[INFO] time: 108.45ms
[INFO] time: 65.08ms
[INFO] time: 57.63ms
[INFO] time: 61.88ms
[INFO] time: 62.15ms
[INFO] time: 65.44ms
[INFO] time: 65.95ms
[INFO] time: 66.79ms
[INFO] time: 72.58ms
[INFO] time: 68.47ms
[INFO] time: 59.03ms
[INFO] time: 60.23ms
[INFO] time: 57.06ms
[INFO] time: 58.45ms
[INFO] time: 66.99ms
[INFO] time: 57.25ms
[INFO] time: 58.15ms
[INFO] time: 66.46ms
[INFO] time: 61.89ms
[INFO] time: 62.08ms
[INFO] time: 67.11ms
[INFO] time: 63.37ms
[INFO] time: 58.18ms
[INFO] time: 64.90ms
[INFO] time: 59.64ms
[INFO] time: 73.22ms
[INFO] time: 62.82ms
[INFO] time: 65.98ms
[INFO] time: 56.74ms
[INFO] time: 64.27ms
[INFO] time: 59.10ms
[INFO] time: 58.85ms
[INFO] time: 66.55ms
[INFO] time: 57.49ms
[INFO] time: 59.54ms
[INFO] time: 61.80ms
[INFO] time: 58.66ms
[INFO] time: 58.11ms
[INFO] time: 74.89ms
[INFO] time: 65.10ms
[INFO] time: 62.36ms
[INFO] time: 64.44ms
[INFO] tim

In [31]:
# savevideo.py

# 모델 데이터 가져오기
#!wget http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2

# 압축 풀기 
#!bzip2 -d /content/shape_predictor_5_face_landmarks.dat.bz2

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

vc = cv2.VideoCapture('love2.mp4')
img_sticker = cv2.imread('king.png')

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
print (vlen) # 비디오 프레임의 총 개수

# writer 초기화
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter('result.mp4', fourcc, 30, (1280, 720))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False:
        break

    start = cv2.getTickCount()
    img_result = img2sticker_orig(img, img_sticker.copy(), detector_hog, landmark_predictor)
    time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
    print ('[INFO] time: %.2fms'%time)

        # 매 프레임 마다 저장합니다.    
    vw.write(cv2.resize(img_result, (1280,720)))

    cv2.imshow('show', img_result)
    key = cv2.waitKey(1)
    if key == 27:
        break

vw.release()
cv2.destroyAllWindows()

54
[INFO] time: 64.12ms
[INFO] time: 145.39ms
[INFO] time: 92.16ms
[INFO] time: 69.15ms
[INFO] time: 63.47ms
[INFO] time: 111.66ms
[INFO] time: 66.19ms
[INFO] time: 66.16ms
[INFO] time: 69.82ms
[INFO] time: 77.57ms
[INFO] time: 85.77ms
[INFO] time: 92.17ms
[INFO] time: 66.13ms
[INFO] time: 63.74ms
[INFO] time: 66.71ms
[INFO] time: 66.55ms
[INFO] time: 74.35ms
[INFO] time: 71.60ms
[INFO] time: 59.29ms
[INFO] time: 65.55ms
[INFO] time: 65.96ms
[INFO] time: 70.26ms
[INFO] time: 58.35ms
[INFO] time: 68.19ms
[INFO] time: 60.65ms
[INFO] time: 58.46ms
[INFO] time: 57.73ms
[INFO] time: 63.64ms
[INFO] time: 58.43ms
[INFO] time: 60.92ms
[INFO] time: 63.04ms
[INFO] time: 65.50ms
[INFO] time: 62.00ms
[INFO] time: 76.14ms
[INFO] time: 59.48ms
[INFO] time: 68.17ms
[INFO] time: 63.69ms
[INFO] time: 68.19ms
[INFO] time: 65.17ms
[INFO] time: 61.68ms
[INFO] time: 57.87ms
[INFO] time: 57.88ms
[INFO] time: 58.49ms
[INFO] time: 59.58ms
[INFO] time: 59.36ms
[INFO] time: 64.72ms
[INFO] time: 65.90ms
[INFO] t